In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
a = 1000000
#rho_0 = 998.2
volume = 1e-4
alpha = np.linspace(0, 3 * np.pi /4, a)

In [3]:
rpm = 160
radius = 0.1
def calculate_rad_velocity(rpm, radius):

    return (rpm/60) * 2 * np.pi * radius

def calculate_angle_velocity(rpm):
    
    return (rpm/60) * 2 * np.pi

In [4]:
viscosity = 1e-3
height = 1.5e-4
#delta_p = 15*10**5
delta_x = 16e-3
width = 0.06556
rad_velocity = calculate_rad_velocity(rpm, radius)
rad_velocity = 0.0
angle_velocity = calculate_angle_velocity(rpm)
t = alpha / angle_velocity
delta_t = t[1] - t[0]
delta_t = 1e-6


In [5]:
def calculate_volume_flow (width, height, rad_velocity, viscosity, delta_p, delta_x):
    
    volume_flow = width * height * (rad_velocity - (2/(3*viscosity))*(height**2) * delta_p / delta_x)
    
    return volume_flow

In [6]:
class Fluid():
    def __init__(self,
                 kinematic_viscosity: float = 1e-6,
                 norm_density: float = 998.2,
                 norm_pressure: float = 101325,
                 initial_pressure: float = 101325,
                 factor_b: float = 3.214e8,
                 factor_m: float = 7.0,
                )-> None:
        """
        Initializes the fluid class with NIST values of fluid properties
        """
        self._kinematic_viscosity = kinematic_viscosity
        self._norm_density = norm_density
        self._norm_pressure = norm_pressure
        self._initial_pressure = initial_pressure
        self._factor_b = factor_b
        self._factor_m = factor_m
        
    @property
    def kinematic_viscosity(self):
        return self._kinematic_viscosity
    
    @property
    def norm_density(self):
        return self._norm_density
    
    @property
    def norm_pressure(self):
        return self._norm_pressure
    
    @property
    def initial_pressure(self):
        return self._initial_pressure
    
    @property
    def factor_b(self):
        return self._factor_b
    
    @property
    def factor_m(self):
        return self._factor_m
    
    def density(self, pressure: float = None, temperature: float = None)-> float:
        pressure_diff = np.asarray(pressure) - self.norm_pressure
        result = self.norm_density * (((pressure_diff / self._factor_b) + 1)**(1/self._factor_m))
        return result
    def pressure(self, density: float = None, temperature: float = None)-> float:
        result = self.norm_pressure + self.factor_b * ((np.asarray(density) / self.norm_density)**self.factor_m - 1)
        return result
        

In [7]:
def calculate_mass_flow(volume_flow, density):
    delta_m = abs(volume_flow) * density
    return delta_m

In [8]:
def calculate_new_density(delta_m, volume, delta_t, density_t0):
    density_t1 = density_t0 + delta_m * delta_t / volume
    return density_t1

In [ ]:
water = Fluid()
rho = np.empty(a)

p0 = 1e5
p1 = 1e5
p2 = 16e5

plog = []
plog.append(p1)

delta_p01 = p1 - p0
delta_p12 = p2 - p1

density0 = water.density(pressure=p0,temperature=None)
density1 = water.density(pressure=p1,temperature=None)
density2 = water.density(pressure=p2,temperature=None)

for i in range(a):
    density1_t0 = density1
    volume_flow01 = calculate_volume_flow(0.065, 1.5e-4, 0.0, 1e-3, delta_p01, 8e-3)
    volume_flow12 = calculate_volume_flow(0.065, 1.5e-4, 0.0, 1e-3, delta_p12, 8e-3)
    
    mass_flow01 = calculate_mass_flow(volume_flow01,density0)
    #print(mass_flow01)
    mass_flow12 = calculate_mass_flow(volume_flow12,density1)
    delta_m = mass_flow12 - mass_flow01
    #delta_m = 0.0
    
    density1 = calculate_new_density(delta_m, volume, delta_t, density1_t0)
    p1 = water.pressure(density=density1)
    plog.append(p1)
    
    delta_p01 = p1 -p0
    delta_p12 = p2- p1

In [ ]:
time = np.linspace(0, (delta_t * a), len(plog))
plt.figure(figsize=(16,10))
plt.plot(time,plog)
plt.xlim(0.0, 1)
plt.ylim(0,10e5)

In [ ]:
print(plog[-1])